# Liionpack demo

Andrew Weng

March 10, 2022

In [ ]:
import liionpack as lp
import pybamm
import numpy as np

## Configure the parameters and define the netlist

In [ ]:

I_mag = 5.0
OCV_init = 4.0  # used for intial guess
Ri_init = 5e-2  # used for intial guess
R_busbar = 1.5e-3
R_connection = 1e-2
Np = 4
Ns = 2
Nbatt = Np * Ns
netlist = lp.setup_circuit(
    Np=Np, Ns=Ns, Rb=R_busbar, Rc=R_connection, Ri=Ri_init, V=OCV_init, I=I_mag
)


In [ ]:
print(netlist)

lp.draw_circuit(netlist, cpt_size=1.0, dpi=150, node_spacing=2.5)



## Set up the experiment

In [ ]:

experiment = pybamm.Experiment(
    [
        "Charge at 5 A for 30 minutes",
        "Rest for 15 minutes",
        "Discharge at 5 A for 30 minutes",
        "Rest for 15 minutes",
    ],
    period="10 seconds",
)



### Initialize the parameter set

In [ ]:

chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)



### Initialize the battery model

In [ ]:
SPMe = pybamm.models.full_battery_models.lithium_ion.SPMe()

In [ ]:
SPMe.variable_names()[:10]

In [ ]:
SPMe.variables.search("X-averaged")

In [ ]:
output_variables = [
    "X-averaged negative particle surface concentration [mol.m-3]",
    "X-averaged positive particle surface concentration [mol.m-3]",
]

## Run the solver

In [ ]:
output = lp.solve(
    netlist=netlist,
    parameter_values=parameter_values,
    experiment=experiment,
    output_variables=output_variables,
    initial_soc=0.5
)

## Plot the outputs

In [ ]:
lp.plot_output(output)
